In [1]:
import os

In [2]:
%pwd

'd:\\Programming\\MLOps\\Practice\\Wine_Quality_Prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Programming\\MLOps\\Practice\\Wine_Quality_Prediction'

In [47]:
# os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/gauravkumaya/Wine_Quality_Prediction.mlflow"
os.environ["MLFLOW_TRACKING_URI"] = "https://127.0.0.1:5000"

# os.environ["MLFLOW_TRACKING_USERNAME"] = "gauravkumaya"

# os.environ["MLFLOW_TRACKING_PASSWORD"] = "783dfda3c54d4b4f200fd0047698a112e4b3c0e6"

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_path: Path
    target_column: str
    mlflow_uri: str
    

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.Elasticnet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params = params,
            metric_file_path=config.metric_file_path,
            target_column=schema.name,
            # mlflow_uri="https://dagshub.com/gauravkumaya/Wine_Quality_Prediction.mlflow"
            mlflow_uri="http://127.0.0.1:5000"
        )
    
        return model_evaluation_config


In [10]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import joblib
import dagshub
from mlflow.models import infer_signature

In [13]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    
    def eval_matrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        
        return rmse, mae, r2
    
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        X_test = test_data.drop(columns=self.config.target_column)
        y_test = test_data[self.config.target_column]
        
        y_pred = model.predict(X_test)
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        mlflow.set_experiment("Wine Quality Prediction")
        signature = infer_signature(X_test, model.predict(X_test))
        
        with mlflow.start_run():
            
            (rmse, mae, r2) = self.eval_matrics(y_test, y_pred)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_path), data=scores)
            
            mlflow.log_params(self.config.all_params)
            
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)
            
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                # Register the model
                # There are other ways to use the Model Registry, which depends on nthe use
                mlflow.sklearn.log_model(sk_model=model, name="model", signature=signature, registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(sk_model=model, name="model", signature=signature)

In [16]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-06-30 01:54:50,870: INFO: common: yaml file: config\config.yaml loaded successfuly]
[2025-06-30 01:54:50,872: INFO: common: yaml file: params.yaml loaded successfuly]
[2025-06-30 01:54:50,875: INFO: common: yaml file: schema.yaml loaded successfuly]
[2025-06-30 01:54:50,878: INFO: common: created directory at: artifacts]
[2025-06-30 01:54:50,880: INFO: common: created directory at: artifacts/model_evlauation]
[2025-06-30 01:54:50,917: INFO: common: json file saved at: artifacts\model_evlauation\metrics.json]


C:\Users\kumay\AppData\Roaming\Python\Python313\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
